# 01) GH Archive 주간 데이터 다운로드
- 1주일(또는 지정 범위) 시간별 JSON.GZ 파일을 다운로드하여 `data/train/raw` 또는 `data/test/raw`에 저장합니다.
- tqdm 진행바는 `ncols=30`으로 고정.

In [1]:

# 기본 설정 (Windows 로컬 경로에 맞게)
BASE_DIR = r"C:\Users\EL040\Desktop\MS_3rd-Project\basemodel"
SPLIT = "train"   # "train" 또는 "test"
START = "2025-08-01T00:00:00Z"  # UTC 기준 시작
END   = "2025-08-07T23:59:59Z"  # UTC 기준 종료

# (선택) 특정 repo/org만 필터링하여 저장하려면 여기서 지정 (None이면 전체 저장)
# 예시: TARGET_ORGS = {"pandas-dev", "pytorch"}  /  TARGET_REPOS = {"pandas-dev/pandas"}
TARGET_ORGS = None
TARGET_REPOS = None


In [ ]:

import os, gzip, json
from datetime import datetime, timezone
from tqdm import tqdm

from common import get_paths, hour_urls, download_one, parse_json_gz, ensure_dir

paths = get_paths(BASE_DIR)
raw_dir = paths["data_train_raw"] if SPLIT == "train" else paths["data_test_raw"]
ensure_dir(raw_dir)

start_utc = datetime.fromisoformat(START.replace("Z", "+00:00")).astimezone(timezone.utc)
end_utc   = datetime.fromisoformat(END.replace("Z", "+00:00")).astimezone(timezone.utc)

print("Download target:", raw_dir)


In [ ]:

# 시간대별 파일을 순회하며 다운로드
for hour_dt, url in hour_urls(start_utc, end_utc):
    out_name = f"{hour_dt.strftime('%Y-%m-%d')}-{hour_dt.hour}.json.gz"
    out_path = os.path.join(raw_dir, out_name)
    _ = download_one(url, out_path)
print("다운로드 완료.")


In [ ]:

# (선택) repo/org 필터링 저장: TARGET_ORGS/TARGET_REPOS가 지정된 경우, 필터링 된 라인만 남긴 별도 파일 생성
if TARGET_ORGS or TARGET_REPOS:
    for fn in os.listdir(raw_dir):
        if not fn.endswith(".json.gz"): 
            continue
        full = os.path.join(raw_dir, fn)
        filtered_path = os.path.join(raw_dir, fn.replace(".json.gz", ".filtered.json.gz"))
        cnt_in, cnt_out = 0, 0
        with gzip.open(filtered_path, "wt", encoding="utf-8") as wf:
            for evt in parse_json_gz(full):
                cnt_in += 1
                repo = (evt.get("repo") or {}).get("name") or ""
                org = repo.split("/")[0] if "/" in repo else ""
                ok = True
                if TARGET_ORGS and org not in TARGET_ORGS:
                    ok = False
                if TARGET_REPOS and repo not in TARGET_REPOS:
                    ok = False
                if ok:
                    wf.write(json.dumps(evt, ensure_ascii=False) + "\n")
                    cnt_out += 1
        print(f"{fn}: in={cnt_in}, out={cnt_out}, -> {os.path.basename(filtered_path)}")
else:
    print("필터링 생략 (전체 저장).")
